<a href="https://colab.research.google.com/github/ravi02512/WNS-link-prediction-with-LightGBM-with-Bayes-hyperparameter-optimization/blob/master/Wns_ad_click_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
%matplotlib inline

In [0]:
item_data=pd.read_csv('/content/gdrive/My Drive/ctr prediction /ctr prediction/train/item_data.csv')
train_data=pd.read_csv('/content/gdrive/My Drive/ctr prediction /ctr prediction/train/train.csv')
log_data=pd.read_csv('/content/gdrive/My Drive/ctr prediction /ctr prediction/train/view_log.csv')

In [0]:
from datetime import datetime

train_data['impression_date'] = train_data['impression_time'].apply(lambda x:datetime.strptime(x, '%Y-%m-%d %H:%M:%S').day)
train_data['impression_hour'] = train_data['impression_time'].apply(lambda x:(datetime.strptime(x, '%Y-%m-%d %H:%M:%S').hour))
train_data['impression_weekday'] = train_data['impression_time'].apply(lambda x:datetime.strptime(x, '%Y-%m-%d %H:%M:%S').weekday())
train_data.drop(['impression_id', 'impression_time'], axis=1, inplace=True)
train_data.head()

,user_id,app_code,os_version,is_4G,is_click,impression_date,impression_hour,impression_weekday
0,87862,422,old,0,0,15,0,3
1,63410,467,latest,1,1,15,0,3
2,71748,259,intermediate,1,0,15,0,3
3,69209,244,latest,1,0,15,0,3
4,62873,473,latest,0,0,15,0,3


In [0]:
# ## creating server freaquency for each user 

# server_visited=pd.DataFrame(log_data.user_id.value_counts())
# server_visited.reset_index(inplace=True)
# server_visited=server_visited.rename(columns={'index':'user_id', 'user_id':'Frequency'})
# train_data=train_data.merge(server_visited, on='user_id')

In [0]:
# bins=[0,100,200,300,400,550]
# groups=['0','1','2','3','4']


# train_data['binned_app_code']=pd.cut(train_data.app_code, bins=bins, labels=groups)
# train_data.head()





In [0]:
# train_data.drop('app_code',axis=1, inplace=True)
train_data=pd.get_dummies(train_data, drop_first=True)
# train_data.head()

In [0]:
train_data.drop('user_id' ,axis=1 , inplace=True)

In [0]:
def create_sample_data(train_data):
  train_1=train_data[train_data['is_click']==1]
  train_0=train_data[train_data['is_click']==0]
  sample_df=train_0.sample(len(train_1))
  sampled_train=pd.concat([train_1, sample_df])
  sampled_train = sampled_train.sample(frac=1).reset_index(drop=True)
  
  X_train = train.loc[:, train.columns != 'is_click']
  y_target = train.is_click.values
  #create lightgbm dataset
  msk = np.random.rand(len(X_train)) < 0.9
  lgb_train = lgb.Dataset(X_train[msk], y_target[msk])
  lgb_eval = lgb.Dataset(X_train[~msk], y_target[~msk], reference=lgb_train)
  
  return sampled_train

In [0]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.0.1-cp36-none-any.whl size=10032 sha256=c630ed74515b96c12e99f2ff6b39d266bc32b2c4a9e120af71829574bbb0aba9
  Stored in directory: /root/.cache/pip/wheels/1d/0d/3b/6b9d4477a34b3905f246ff4e7acf6aafd4cc9b77d473629b77
Successfully built bayesian-optimization


In [0]:
### Hyperparameter optimization 
import warnings
import time
warnings.filterwarnings("ignore")
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score

In [0]:
train=create_sample_data(train_data)
train.head()

,app_code,is_4G,is_click,impression_date,impression_hour,impression_weekday,os_version_latest,os_version_old
0,296,1,0,26,9,0,0,0
1,213,1,0,4,19,1,1,0
2,469,1,1,4,22,1,0,0
3,207,0,0,15,7,3,0,1
4,127,1,0,1,0,5,1,0


In [0]:
from sklearn.preprocessing import LabelEncoder
def label_encoder(input_df, encoder_dict=None):
    """ Process a dataframe into a form useable by LightGBM """
    # Label encode categoricals
    categorical_feats = input_df.columns[input_df.dtypes == 'object']
    for feat in categorical_feats:
        encoder = LabelEncoder()
        input_df[feat] = encoder.fit_transform(input_df[feat].fillna('NULL'))
    return input_df, categorical_feats.tolist(), encoder_dict
# application_train, categorical_feats, encoder_dict = label_encoder(train_data)

X =train.drop('is_click', axis=1)
y =train.is_click


In [0]:
def bayes_parameter_opt_lgb(X, y, init_round=10, opt_round=25, n_folds=5, random_seed=6, n_estimators=10000, learning_rate=0.001, output_process=False):
    # prepare data
    train_data = lgb.Dataset(data=X, label=y,free_raw_data=False)
    # parameters
    def lgb_eval(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight):
        params = {'application':'binary','num_iterations': n_estimators, 'learning_rate':learning_rate, 'early_stopping_round':100, 'metric':'auc'}
        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['lambda_l1'] = max(lambda_l1, 0)
        params['lambda_l2'] = max(lambda_l2, 0)
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, verbose_eval =200, metrics=['auc'])
        return max(cv_result['auc-mean'])

        
    lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (50, 70),
                                            'feature_fraction': (0.4, 0.8),
                                            'bagging_fraction': (0.7, 1),
                                            'max_depth': (8, 15),
                                            'lambda_l1': (0.1, 2),
                                            'lambda_l2': (10, 20),
                                            'min_split_gain': (0.01, 0.9),
                                            'min_child_weight': (10, 20)}, random_state=0)
    
    # optimize
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    
    # output optimization process
    if output_process==True: lgbBO.points_to_csv("bayes_opt_result.csv")
    
    # return best parameters
    return lgbBO.max

opt_params = bayes_parameter_opt_lgb(X, y, init_round=5, opt_round=20, n_folds=5, random_seed=6, n_estimators=100, learning_rate=0.1)

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  0.6868   |  0.8646   |  0.6861   |  1.245    |  15.45    |  10.97    |  16.46    |  0.3995   |  67.84    |
|  2        |  0.6891   |  0.9891   |  0.5534   |  1.604    |  15.29    |  11.98    |  19.26    |  0.07322  |  51.74    |
|  3        |  0.6868   |  0.7061   |  0.733    |  1.578    |  18.7     |  14.85    |  17.99    |  0.4207   |  65.61    |
|  4        |  0.6883   |  0.7355   |  0.656    |  0.3724   |  19.45    |  11.65    |  14.15    |  0.2455   |  65.48    |
|  5        |  0.6871   |  0.8368   |  0.6274   |  0.1357   |  16.18    |  12.28    |  16.17    |  0.8499   |  63.64    |
|  6        |  0.6878   |  0.9824   |  0.5688   |  0.3564   |  10.58    |  8.048    |  10.25    |  0.8343   |  50.18    |
|  7        |  0.6873   

In [0]:
params=opt_params['params']
params['boosing_type']='gbdt'
params['objective']='binary'
params['metric']={'auc'}
params['num_leaves']=int(params['num_leaves'])
params['max_depth']=int(params['max_depth'])

In [0]:
# params={'boosting_type': 'gbdt',
#  'objective': 'binary',
#  'metric': {'auc'},
#  'bagging_fraction': 0.8925977756858038,
#  'feature_fraction': 0.7409826754430792,
#  'lambda_l1': 0.1819868252742174,
#  'lambda_l2': 10.18927603243362,
#  'max_depth': 14,
#  'min_child_weight': 10.593669006692064,
#  'min_split_gain': 0.31810578262107553,
#  'num_leaves': 69}

params

{'bagging_fraction': 0.9724129460559441,
 'boosing_type': 'gbdt',
 'feature_fraction': 0.565961669766817,
 'lambda_l1': 0.5171748517693862,
 'lambda_l2': 19.772798347274296,
 'max_depth': 8,
 'metric': {'auc'},
 'min_child_weight': 10.459950175260222,
 'min_split_gain': 0.3426450785158826,
 'num_leaves': 50,
 'objective': 'binary'}

In [0]:
X_train = train.loc[:, train.columns != 'is_click']
y_target = train.is_click.values
#create lightgbm dataset
msk = np.random.rand(len(X_train)) < 0.9
lgb_train = lgb.Dataset(X_train[msk], y_target[msk])
lgb_eval = lgb.Dataset(X_train[~msk], y_target[~msk], reference=lgb_train)


In [0]:
import lightgbm as lgb


def create_model(lgb_train, lgb_eval, params):

  

  print('Start training...')

  # train

  gbm = lgb.train(params,

                  lgb_train,

                  num_boost_round=4000,

                  valid_sets=lgb_eval,

                  early_stopping_rounds=500)

  

  return gbm


In [0]:
model_dict={}
for i in range(20):
  
  model_dict['model'+str(i)]=create_model(train)

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=4000,
                valid_sets=lgb_eval,
                early_stopping_rounds=500)

[1]	valid_0's auc: 0.513146
Training until validation scores don't improve for 500 rounds.
[2]	valid_0's auc: 0.518395
[3]	valid_0's auc: 0.700519
[4]	valid_0's auc: 0.705687
[5]	valid_0's auc: 0.707929
[6]	valid_0's auc: 0.707619
[7]	valid_0's auc: 0.708023
[8]	valid_0's auc: 0.708768
[9]	valid_0's auc: 0.708917
[10]	valid_0's auc: 0.709454
[11]	valid_0's auc: 0.709547
[12]	valid_0's auc: 0.709913
[13]	valid_0's auc: 0.709575
[14]	valid_0's auc: 0.708816
[15]	valid_0's auc: 0.708776
[16]	valid_0's auc: 0.708774
[17]	valid_0's auc: 0.70875
[18]	valid_0's auc: 0.708795
[19]	valid_0's auc: 0.708641
[20]	valid_0's auc: 0.708211
[21]	valid_0's auc: 0.708069
[22]	valid_0's auc: 0.708186
[23]	valid_0's auc: 0.708071
[24]	valid_0's auc: 0.708362
[25]	valid_0's auc: 0.708571
[26]	valid_0's auc: 0.708373
[27]	valid_0's auc: 0.708858
[28]	valid_0's auc: 0.708761
[29]	valid_0's auc: 0.709008
[30]	valid_0's auc: 0.708698
[31]	valid_0's auc: 0.708587
[32]	valid_0's auc: 0.708522
[33]	valid_0's auc:

In [0]:
test_data=pd.read_csv('/content/gdrive/My Drive/ctr prediction /ctr prediction/test/test.csv')
test_data['impression_date'] = test_data['impression_time'].apply(lambda x:datetime.strptime(x, '%Y-%m-%d %H:%M:%S').day)
test_data['impression_hour'] = test_data['impression_time'].apply(lambda x:(datetime.strptime(x, '%Y-%m-%d %H:%M:%S').hour))
test_data['impression_weekday'] = test_data['impression_time'].apply(lambda x:datetime.strptime(x, '%Y-%m-%d %H:%M:%S').weekday())
test_data.drop(['impression_id', 'impression_time'], axis=1, inplace=True)
# test_data=test_data.merge(server_visited, on='user_id')
test_data=pd.get_dummies(test_data, drop_first=True)
test_data.head()

,user_id,app_code,is_4G,impression_date,impression_hour,impression_weekday,os_version_latest,os_version_old
0,44754,127,1,13,7,3,1,0
1,29656,44,0,13,7,3,1,0
2,25234,296,1,13,7,3,1,0
3,22988,207,1,13,7,3,1,0
4,35431,242,1,13,7,3,1,0


In [0]:
l=gbm.predict(test_data)
submission_data=pd.read_csv('/content/gdrive/My Drive/ctr prediction /ctr prediction/sample_submission.csv')
submission_data['is_click']=[1 if i>=0.5 else 0 for i in l]
submission_data.head()

,impression_id,is_click
0,a9e7126a585a69a32bc7414e9d0c0ada,0
1,caac14a5bf2ba283db7708bb34855760,0
2,13f10ba306a19ce7bec2f3cae507b698,0
3,39c4b4dc0e9701b55a0a4f072008fb3f,0
4,bf5a572cca75f5fc67f4b14e58b11d70,0


In [0]:
import os
os.chdir('/content/gdrive/My Drive/')
submission_data.to_csv('sub8_file.csv', index=False)